#  Connectivity stuff ..

In [ ]:
#  From,
#
#     https://github.com/KatanaGraph/katana-enterprise/blob/master/python/test/notebooks/red/analytics_remote.ipynb


In [14]:

from katana import remote
from katana.remote import import_data

from tabulate import tabulate


my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)


# Libraries specfic to Analytics ..

In [ ]:

import numpy as np
from katana import remote
from katana.remote import analytics, import_data
from katana_enterprise.distributed import PartitionPolicy

print("--")


# Actual analytics

In [8]:

l_cntr = 20                       #  Used below

print("--")


--


In [9]:

# l_source              = 2
l_source              = "2"


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source,
      #
   weight_property      = "LENGTH",
   result_property_name = l_output_propname,
)

print("--")


          0/? [?op/s]

--


In [15]:

l_result1 = my_graph.query(
    """
    MATCH (n) 
    RETURN n 
    """
   )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
    """
    MATCH (n) - [r] -> (m)
    RETURN r 
    """
   )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+------------------+------------------+----------+
|    |   n.internal_id | n.labels    | n.airport_code   | n.airport_name   | n.type   |
|----+-----------------+-------------+------------------+------------------+----------|
|  0 |               0 | ['Airport'] | SJC              | San Jose         | node     |
|  1 | 281474976710658 | ['Airport'] | DEN              | Denver           | node     |
|  2 | 281474976710656 | ['Airport'] | MKE              | Milwaukee        | node     |
|  3 | 281474976710657 | ['Airport'] | ORD              | Chicago O-Hare   | node     |
+----+-----------------+-------------+------------------+------------------+----------+


          0/? [?op/s]

+----+-----------------+----------------+-----------------+-----------+------------+-----------------+------------------+----------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.LENGTH |      r.start.id | r.start.labels   | r.type   |
|----+-----------------+----------------+-----------------+-----------+------------+-----------------+------------------+----------|
|  0 | 281474976710657 | ['Airport']    |               1 | FLIES_TO  |       1829 |               0 | ['Airport']      | edge     |
|  1 | 281474976710658 | ['Airport']    |               0 | FLIES_TO  |        948 |               0 | ['Airport']      | edge     |
|  2 | 281474976710656 | ['Airport']    | 281474976710657 | FLIES_TO  |         66 | 281474976710657 | ['Airport']      | edge     |
|  3 | 281474976710658 | ['Airport']    | 281474976710658 | FLIES_TO  |        886 | 281474976710657 | ['Airport']      | edge     |
|  4 | 281474976710657 | ['Airport']    | 281474976710659 | FLIES_TO 